In [4]:
import requests
import pandas as pd
import numpy as np
import matplotlib as plt
api_key = "WFHZHJATQIO0HDH7"

In [5]:
def get_stocks(equity):
    base_url = "https://www.alphavantage.co/query?"
    url = base_url + 'function=TIME_SERIES_INTRADAY' + '&symbol=' + equity + '&interval=60min&apikey=' + api_key
    response = requests.get(url)
    if response.status_code == 200:
        df = pd.DataFrame(response.json())
        return df
    else:
        print("Could not fetch data")
        return

In [6]:
def column_summary(df):
    summary = []

    for col_name in df.columns:
        col_dtype = df[col_name].dtype
        num_of_nulls = df[col_name].isnull().sum()
        num_of_non_nulls = df[col_name].notnull().sum()

        summary.append( {
            'col_name' : col_name,
            'num_of_nulls' : num_of_nulls,
            'num_of_non_nulls' : num_of_non_nulls,
        })

    summary_df = pd.DataFrame(summary)
    return summary_df

In [7]:
def get_time_series_data(df):
    time_data = []
    for i, col_name in enumerate(df.columns):
        if i > 6 :
            time_data.append(df[col_name])
        else:
            pass
    time_series_data = pd.DataFrame(time_data)
    return time_series_data

In [8]:
def main():
    equity = input("Enter the equity of choice: ").strip()
    raw_stocks = get_stocks(equity)
    time_series_data = get_time_series_data(raw_stocks)
    summary = column_summary(raw_stocks)
    print(raw_stocks)
    display(summary)
    print(time_series_data)

In [9]:
if __name__ == "__main__":
    main()

Enter the equity of choice:  IBM 


                                                             Meta Data  \
1. Information       Intraday (60min) open, high, low, close prices...   
2. Symbol                                                          IBM   
3. Last Refreshed                                  2025-02-21 19:00:00   
4. Interval                                                      60min   
5. Output Size                                                 Compact   
...                                                                ...   
2025-02-13 04:00:00                                                NaN   
2025-02-12 19:00:00                                                NaN   
2025-02-12 18:00:00                                                NaN   
2025-02-12 17:00:00                                                NaN   
2025-02-12 16:00:00                                                NaN   

                                                   Time Series (60min)  
1. Information                        

,col_name,num_of_nulls,num_of_non_nulls
0,Meta Data,100,6
1,Time Series (60min),6,100


Empty DataFrame
Columns: []
Index: []
